In [1]:
import multiprocessing
from multiprocessing.pool import ThreadPool
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import StarmapParallelization
from pymoo.constraints.as_obj import ConstraintsAsObjective
from pymoo.constraints.as_penalty import ConstraintsAsPenalty
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize

import optimize as op

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


In [2]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 1.6
opt.getLiftoffState()
print("Initial Cost", opt.objective(opt.bp_init_guess))


class CorgiProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        super().__init__(n_var=24,
                         n_obj=1,
                         n_ieq_constr=24,
                         xl=np.array([0.33, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.33, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.33, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.33, 0.00, 0.01, -0.05, 0.01, -0.05]),
                         xu=np.array([0.35, 0.02, 1, 0.5, 1, 0.5,
                                      0.35, 0.02, 1, 0.5, 1, 0.5,
                                      0.35, 0.02, 1, 0.5, 1, 0.5,
                                      0.35, 0.02, 1, 0.5, 1, 0.5]),
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        f  = opt.objective(x)

        g1 = -1 * opt.innerWsPFConstraint(x)
        g2 = -1 * opt.outerWsPFConstraint(x)
        g3 = np.abs(opt.velocityConstraint(x)) - 0.01
        print("g3", np.average(g3))
        g = np.hstack((g1, g2))
        g = np.hstack((g, g3))

        out["F"] = [f]
        out["G"] = list(g)

# initialize the thread pool and create the runner
# n_proccess = 8
# pool = multiprocessing.Pool(n_proccess)
# runner = StarmapParallelization(pool.starmap)

# initialize the thread pool and create the runner
n_threads = 16
pool = ThreadPool(n_threads)
runner = StarmapParallelization(pool.starmap)
problem = CorgiProblem(elementwise_runner=runner, elementwise_evaluation=True)
# problem = CorgiProblem()


[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
iter 1 cost -6.399850586749445
Initial Cost -6.399850586749445


In [3]:
algorithm = NSGA2(
    pop_size=60,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)
# algorithm = PSO()

In [4]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 100)

In [5]:

result = minimize(ConstraintsAsPenalty(problem, 1.0),
                  algorithm,
                  termination,
                  seed=1,
                  save_history=False,
                  verbose=False)

pool.close()


[0.33834044 0.01440649 0.01011323 0.11628291 0.15528833 0.00078623]
[0.3337252  0.00691121 0.4027998  0.2463492  0.42500257 0.32687073]
[0.33408904 0.01756235 0.03711372 0.31875713 0.42313175 0.25727941]
[0.33280774 0.00396203 0.80273712 0.48254387 0.32028994 0.33077744]
[ 0.34752778  0.01789213  0.09419377 -0.02851987  0.17813212  0.43297838]
[0.33196694 0.00842215 0.95831063 0.24324091 0.69495834 0.1235336 ]
[0.34373002 0.01669251 0.02810539 0.36257937 0.98897248 0.36149111]
[0.33560888 0.01578559 0.11219375 0.19634144 0.90950955 0.11148778]
[0.33575551 0.00260057 0.02917329 0.32335954 0.21951183 0.09605066]
[0.33983146 0.00106725 0.57837643 0.03070072 0.59341248 0.3348671 ]
[0.33204669 0.00828112 0.69745616 0.1777986  0.05945392 0.24474302]
[0.34327589 0.01029778 0.94514881 0.27260527 0.9043679  0.02561109]
[0.33278553 0.01614783 0.40370007 0.04094481 0.92823349 0.14127122]
[0.34501624 0.01451996 0.88447303 0.29301971 0.75343301 0.14189409]
[0.33539856 0.01791772 0.43381028 0.480662

iter 2 cost 1205.597783189119
iter 3 cost 947.0443739066969
iter 4 cost 1310.161438835297
iter 5 cost 1215.0341063720505
iter 6 cost 1403.5134481714692
iter 7 cost 824.5953562432268
iter 8 cost 1090.7656577152231
iter 9 cost 913.7907773624069
iter 10 cost 936.935731442667
iter 11 cost 1028.4897189606634
iter 12 cost 1181.664465146066
iter 13 cost 894.8173602921848
iter 14 cost 1208.488003960853
g3 2.021811705392352
[0.33495466 0.00524346 0.75267069 0.20133643 0.06636014 0.22968393]
[ 0.3342392   0.01597208  0.30435807 -0.03481669  0.59749812  0.41411224]
[0.33762032 0.01499717 0.51603006 0.24752349 0.95983998 0.39217849]
[0.33064646 0.01418775 0.47035147 0.47115192 0.22921841 0.09688961]
iter 15 cost 895.9243976346262
iter 16 cost 1147.7014866677316
g3 1.7495346421840914
[0.33162948 0.00857238 0.11792858 0.29858272 0.80493361 0.33324027]
[0.34532423 0.00684908 0.84739297 0.18582283 0.82576977 0.29457289]
[ 0.33286846  0.00156774  0.02814932 -0.01330125  0.46399798  0.01233806]
[0.33055

In [6]:
print(result.X)
print(result.F)
min_ = np.min(result.F, axis=0)
opt.exportBezierData(result.X)

[ 3.30000835e-01  3.14882469e-05  1.01953266e-02 -4.99180125e-02
  7.91782749e-01  1.84267033e-01  3.30002059e-01  2.85944905e-04
  1.00037062e-02 -4.99941037e-02  3.95703359e-01  3.99071499e-01
  3.30028312e-01  2.80395383e-05  1.66741915e-01  8.25659229e-02
  1.02762374e-02 -4.99991785e-02  3.30060681e-01  1.83734290e-03
  1.18377549e-01  2.15902539e-01  1.01183910e-02 -4.95125705e-02]
[91.71456187]
data export
sbrio exported to ./csv_trajectory/20230828/2023-08-30_traj_400_6.csv
data export
webot exported /home/guanlunlu/corgi_webots/controllers/supervisor/2023-08-30_traj_400_6.csv


In [7]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.6
opt.getLiftoffState()
print("Initial Cost", opt.objective(result.X))

[ 3.30000835e-01  3.14882469e-05  1.01953266e-02 -4.99180125e-02
  7.91782749e-01  1.84267033e-01]
[ 3.30002059e-01  2.85944905e-04  1.00037062e-02 -4.99941037e-02
  3.95703359e-01  3.99071499e-01]
[ 3.30028312e-01  2.80395383e-05  1.66741915e-01  8.25659229e-02
  1.02762374e-02 -4.99991785e-02]
[ 0.33006068  0.00183734  0.11837755  0.21590254  0.01011839 -0.04951257]
iter 1 cost 110.57432878727099
Initial Cost 110.57432878727099
